<a href="https://colab.research.google.com/github/erberry/ThinkML/blob/main/convert_and_quantize_chinese_llama_and_alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# 转换并量化中文LLaMA和Alpaca模型

项目地址：https://github.com/ymcui/Chinese-LLaMA-Alpaca

⚠️ 内存消耗提示（确保刷出来的机器RAM大于以下要求）：
- 7B模型：15G+
- 13B模型：18G+
- 33B模型：22G+

💡 提示和小窍门：
- 免费用户默认的内存只有12G左右，不足以转换模型。**实测选择TPU的话有机会随机出35G内存**，建议多试几次
- Pro(+)用户请选择 “代码执行程序” -> “更改运行时类型” -> “高RAM”
- 程序莫名崩掉或断开连接就说明内存爆了
- 如果选了“高RAM”之后内存还是不够大的话，选择以下操作，有的时候会分配出很高内存的机器，祝你好运😄！
    - 可以把GPU或者TPU也选上（虽然不会用到）
    - 选GPU时，Pro(+)用户可选“A100”类型GPU

*温馨提示：用完之后注意断开运行时，选择满足要求的最低配置即可，避免不必要的计算单元消耗（Pro只给100个计算单元）。*

## 安装相关依赖

In [ ]:
!pip install torch==1.13.1
!pip install transformers==4.30.2
!pip install peft==0.3.0
!pip install sentencepiece

## 克隆目录和代码

In [ ]:
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'Chinese-LLaMA-Alpaca'...
remote: Enumerating objects: 1927, done.
remote: Counting objects: 100% (496/496), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1927 (delta 340), reused 416 (delta 314), pack-reused 1431
Receiving objects: 100% (1927/1927), 22.93 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (1159/1159), done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 4187, done.
remote: Counting objects: 100% (1772/1772), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 4187 (delta 1677), reused 1599 (delta 1580), pack-reused 2415
Receiving objects: 100% (4187/4187), 3.63 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (2833/2833), done.


## 合并模型（以Alpaca-13B为例）

此处使用的是🤗模型库中提供的基模型（已是HF格式），而不是Facebook官方的LLaMA模型，因此略去将原版LLaMA转换为HF格式的步骤。
**这里直接运行第二步：合并LoRA权重**，生成全量模型权重。可以直接指定🤗模型库的地址，也可以是本地存放地址。
- 基模型：`elinas/llama-13b-hf-transformers-4.29` *（use at your own risk，我们比对过SHA256和正版一致，但你应确保自己有权使用该模型）*
- LoRA模型：`ziqingyang/chinese-alpaca-lora-13b`
   - 如果是Alpaca-Plus模型，记得要同时传入llama和alpaca的lora，教程：[这里](https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/手动模型合并与转换#多lora权重合并适用于chinese-alpaca-plus)
- 输出格式：可选pth或者huggingface，这里选择pth，因为后面要用llama.cpp量化

由于要下载模型，所以需要耐心等待一下，尤其是33B模型。
转换好的模型存放在`alpaca-combined`目录。
如果你不需要量化模型，那么到这一步就结束了，可自行下载或者转存到Google Drive。

In [ ]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora_low_mem.py \
    --base_model 'elinas/llama-13b-hf-transformers-4.29' \
    --lora_model 'ziqingyang/chinese-alpaca-lora-13b' \
    --output_type pth \
    --output_dir alpaca-combined

## 比对SHA256

完整值：https://github.com/ymcui/Chinese-LLaMA-Alpaca/blob/main/SHA256.md

其中本示例生成的Alpaca-13B的标准SHA256：
- 30cefb5be9091c3e17fbba5d91bf16266a2ddf86cde53370a9982b232ff8a2f4
- ce946742b0f122f472e192c3f77d506e0c26578b4b881d07d919553333affecd

使用下述命令评测后发现两者相同，合并无误。

In [ ]:
!sha256sum alpaca-combined/consolidated.*.pth

30cefb5be9091c3e17fbba5d91bf16266a2ddf86cde53370a9982b232ff8a2f4  alpaca-combined/consolidated.00.pth
ce946742b0f122f472e192c3f77d506e0c26578b4b881d07d919553333affecd  alpaca-combined/consolidated.01.pth


## 量化模型
接下来我们使用[llama.cpp](https://github.com/ggerganov/llama.cpp)工具对上一步生成的全量版本权重进行转换，生成4-bit量化模型。

### 编译工具

首先对llama.cpp工具进行编译。

In [ ]:
!cd llama.cpp && make

I llama.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -std=c11   -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wdouble-promotion -Wshadow -Wstrict-prototypes -Wpointer-arith -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I CXXFLAGS: -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS
I LDFLAGS:  
I CC:       cc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
I CXX:      g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0

g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS -c llama.cpp -o llama.o
g++ -I. -I./examples -O3 -std=c++11 -fPIC -DNDEBUG -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wno-multichar -pthread -march=native -mtune=native -DGGML_USE_K_QUANTS exampl

### 模型转换为ggml格式（FP16）

这一步，我们将模型转换为ggml格式（FP16）。
- 在这之前需要把`alpaca-combined`目录挪个位置，把模型文件放到`llama.cpp/zh-models/13B`下，把`tokenizer.model`放到`llama.cpp/zh-models`
- tokenizer在哪里？
    - `alpaca-combined`目录下有
    - 或者从以下网址下载：https://huggingface.co/ziqingyang/chinese-alpaca-lora-7b/resolve/main/tokenizer.model （注意，Alpaca和LLaMA的`tokenizer.model`不能混用！）

💡 转换13B/33B模型提示：
- tokenizer可以直接用7B的，13B/33B和7B的相同
- Alpaca和LLaMA的`tokenizer.model`不能混用！
- 以下看到13B字样的都是文件夹名，与转换过程没有关系了，改不改都行

In [ ]:
!cd llama.cpp && mkdir zh-models && mv ../alpaca-combined zh-models/13B
!mv llama.cpp/zh-models/13B/tokenizer.model llama.cpp/zh-models/
!ls llama.cpp/zh-models/

In [ ]:
!cd llama.cpp && python convert.py zh-models/13B/

### 将FP16模型量化为4-bit

我们进一步将FP16模型转换为4-bit量化模型，此处选择的是新版Q4_K方法。

In [ ]:
!cd llama.cpp && ./quantize ./zh-models/13B/ggml-model-f16.bin ./zh-models/13B/ggml-model-q4_K.bin q4_K

### （可选）测试量化模型解码
至此已完成了所有转换步骤。
我们运行一条命令测试一下是否能够正常加载并进行对话。

FP16和Q4量化文件存放在./llama.cpp/zh-models/7B下，可按需下载使用。

In [ ]:
!cd llama.cpp && ./main -m ./zh-models/13B/ggml-model-q4_K.bin --color -p "详细介绍一下北京的名胜古迹：" -n 128

main: build = 776 (55dbb91)
main: seed  = 1688396239
llama.cpp: loading model from ./zh-models/13B/ggml-model-q4_K.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 49954
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 15 (mostly Q4_K - Medium)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.09 MB
llama_model_load_internal: mem required  = 9607.28 MB (+ 1608.00 MB per state)
llama_new_context_with_model: kv self size  =  400.00 MB

system_info: n_threads = 20 / 40 | AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WAS

In [ ]:
!cd llama.cpp && cp ./zh-models/tokenizer.model ./zh-models/13B/

In [ ]:
!cd llama.cpp/zh-models/13B && ls -alh

total 57G
drwxr-xr-x 2 root root 4.0K Jul  3 15:04 .
drwxr-xr-x 3 root root 4.0K Jul  3 14:36 ..
-rw-r--r-- 1 root root  13G Jul  3 14:30 consolidated.00.pth
-rw-r--r-- 1 root root  13G Jul  3 14:32 consolidated.01.pth
-rw-r--r-- 1 root root  25G Jul  3 14:54 ggml-model-f16.bin
-rw-r--r-- 1 root root 7.4G Jul  3 14:57 ggml-model-q4_K.bin
-rw-r--r-- 1 root root  101 Jul  3 14:28 params.json
-rw-r--r-- 1 root root   96 Jul  3 14:28 special_tokens_map.json
-rw-r--r-- 1 root root  727 Jul  3 14:28 tokenizer_config.json
-rw-r--r-- 1 root root 741K Jul  3 15:04 tokenizer.model


保存到google 网盘

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/llama.cpp/zh-models/13B/ggml-model-q4_K.bin /content/drive/MyDrive/zh-model/chinese-alpaca-13b-q4

In [ ]:
!ls /content/drive/MyDrive/zh-model/chinese-alpaca-13b-q4

/content/drive/MyDrive/zh-model/chinese-alpaca-13b-q4
